# Assignment 2 - part-of-speech prediction from limited context

In this assignment, you will train classifiers that attempt, within a window of five words, to make a binary prediction about whether the third word belongs to a given part of speech (noun, verb, adjective, adverb), but using very limited information -- that is, the last two letters of the first, second, fourth, and fifth word of the sequence, and no information whatsoever directly from the third word itself.  You will strip out all punctuation (using the NLTK `WordPunctTokenizer`), lowercase, and remove stop words (using the NLTK English stop words list).

In other words, you will predict over samples that have two classes, P and not-P, where P is the selected part of speech to classify.  For example, from the sentence, "The quick brown fox jumped over the lazy dog.", we can select the following 5-word windows without stop words, "brown fox jumped lazy dog" and "quick brown fox jumped lazy".  If we select verbs as the part-of-speech we are classifying over, we get the instances <(wn,ox,zy,og),1>, since "jumped" is a verb, but <(ck,wn,ed,zy),0> because "fox" in that context is not.

This means that you will need to take into account the position of the last-two-letter feature:  "zy" as the fourth word's last two letters is different from "zy" as the fifth word's last two letters.  They are two features, say, `zy_4` and `zy_5`.

This will likely not actually work.  But it might!

You will create training and testing samples according to this procedure, and you will build a data structure that can be fed to a support vector machine (SVM) classifier.  You will train the classifier on the training data and evaluate it on the testing data. 

The work will be done in a .py module file in the same folder as this notebook.  **No modifications to this notebook will be graded.** We will run your module using this notebook or one we modify that you won't see in order to test your code.

The file you must create and add to the github repo is `mycode.py`, which will be imported here.  You can create your own notebooks or scripts to test it.  You can put any number of your own helper functions and also put optional parameters on any of the python functions mentioned here. You should also create a Markdown file, `notes.md`, to keep any **concise** notes and remarks about the assignment.  The code must run on mltgpu.

**This assignment is due Monday, 2022 March 7, at 23:59. There are 33 points and 5 bonus points.**

In [1]:
import mycode as mc

## Part 0 - preparation (2 points)

Fork this repository and create and add `mycode.py` and `notes.md`. 

## Part 1 - obtaining the text (3 points)

You will randomly select the given number of lines from the gzipped file we give you (so you will have to figure out how to access gzipped text files).  Explain how you implemented the random selection in `notes.md`. When we run it, it should give a new sample every time.

In [2]:
sampled_lines = mc.sample_lines("/scratch/UN-english.txt.gz", lines=100000)

print(len(sampled_lines))
sampled_lines[40000:40010]

100000


["For reasons stated in paragraph 58 of the Summary, the Panel makes no recommendation with respect to Jiangsu's claim for interest.",
 '99. MONUC will also actively support the opening up of the internal waterway system, including through the deployment of the Uruguayan riverine unit as well as a second riverine unit of about 200 men to be deployed subsequently.',
 'The plane, an Ilyushin-62, carried the registration number TL-ACL.',
 'The package deal also included contract work for pilots of the Moldovan air force, another aspect that was considered beneficial to the Moldovan Ministry of Defence.',
 'Done at Dar es Salaam on the 24th day of June 2001.',
 'The degree of filling shall not exceed 98%.',
 "Based on its findings regarding Lurgi's claim, the Panel recommends no compensation. hoechst ceramtec ag",
 'Burkina Faso',
 'Prepared by the Secretariat',
 'H.E. Mr. Gelson Fonseca, Jr.']

## Part 2 - creating the samples (7 points)

From the sampled lines, you will then randomly create the five-word samples.

You will tokenize the sentences and apply POS-tagging to them -- you need to do this before you create the samples, since POS-tagging needs context. You will then remove stop words and punctuation and lowercase the remainder.  Next, you will randomly, over the entire set of sentences, choose samples of five words in sequence, up to a certain limit.  You find the last two characters of the first, second, fourth, and fifth words, and create the type of structure specified up in the introduction to this assignment for each sample. The exact representation is up to you.

In [3]:
processed_sentences = mc.process_sentences(sampled_lines)
processed_sentences[40000:40010]

[[('reasons', 'NNS'),
  ('stated', 'VBN'),
  ('paragraph', 'NN'),
  ('58', 'CD'),
  ('summary', 'NNP'),
  ('panel', 'NNP'),
  ('makes', 'VBZ'),
  ('recommendation', 'NN'),
  ('respect', 'NN'),
  ('jiangsu', 'NNP'),
  ("'s", 'POS'),
  ('claim', 'NN'),
  ('interest', 'NN')],
 [('99', 'CD'),
  ('monuc', 'NNP'),
  ('also', 'RB'),
  ('actively', 'RB'),
  ('support', 'VB'),
  ('opening', 'NN'),
  ('internal', 'JJ'),
  ('waterway', 'NN'),
  ('system', 'NN'),
  ('including', 'VBG'),
  ('deployment', 'NN'),
  ('uruguayan', 'NNP'),
  ('riverine', 'NN'),
  ('unit', 'NN'),
  ('well', 'RB'),
  ('second', 'JJ'),
  ('riverine', 'NN'),
  ('unit', 'NN'),
  ('200', 'CD'),
  ('men', 'NNS'),
  ('deployed', 'VBN'),
  ('subsequently', 'RB')],
 [('plane', 'NN'),
  ('ilyushin-62', 'NNP'),
  ('carried', 'VBD'),
  ('registration', 'NN'),
  ('number', 'NN'),
  ('tl-acl', 'NNP')],
 [('package', 'NN'),
  ('deal', 'NN'),
  ('also', 'RB'),
  ('included', 'VBD'),
  ('contract', 'NN'),
  ('work', 'NN'),
  ('pilots', '

In [4]:
all_samples = mc.create_samples(processed_sentences, limit=50000)

print(all_samples[25000:25010])

[('ns_1', 'ed_2', 'ns_3', 'em_5', 0), ('ed_1', 'ns_2', 'em_3', 'ay_5', 0), ('ns_1', 'em_2', 'ay_3', 'st_5', 0), ('em_1', 'ay_2', 'st_3', 'ry_5', 0), ('ay_1', 'st_2', 'ry_3', 'ns_5', 0), ('st_1', 'ry_2', 'ns_3', 'al_5', 0), ('ry_1', 'ns_2', 'al_3', 'rt_5', 0), ('42_1', 'al_2', 'al_3', 'on_5', 0), ('al_1', 'al_2', 'on_3', 'ed_5', 0), ('al_1', 'on_2', 'ed_3', 'st_5', 1)]


## Part 3 - convert the samples into a Pandas DataFrame (10 points)

Here, you will take the samples and create a table whose columns are the features and the class and whose rows are the samples.  All the features and the class will be binary.  Note that there may be many columns, in the hundreds or thousands depending on the diversity of the final two consonants of the non-stop-words in the dataset, but the sum of all rows will be five.

In [5]:
fulldf = mc.create_df(all_samples)
fulldf[25000:25010]

,ur_1,ch_2,el_3,ed_5,ch_1,el_2,ed_3,al_5,el_1,ed_2,...,r/_3,r/_2,r/_1,jo_5,ug_5,dm_5,dm_3,eo_5,km_1,VBN
25000,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
25001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25005,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
25006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25008,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25009,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Part 4 - extract training and testing sets (3 points)

Here, you will create the training and testing datasets in order to train the model.  This will be based on a test percentage.  Round up if the percentage does not divide evenly into the sample size.  You will need to separate the class column into the y-values for the classifier.

In [6]:
train_X, train_y, test_X, test_y = mc.split_samples(fulldf, test_percent=20)
len(train_X), len(train_y), len(test_X), len(test_y)

(40000, 40000, 10000, 10000)

## Part 5 - train models (3 points)

You will then train and return two support vector machine (SVM) models using the sklearn SVC class.  You should allow a choice between linear and radial basis function kernels.

In [7]:
model_linear = mc.train(train_X, train_y, kernel='linear')
model_rbf = mc.train(train_X, train_y, kernel="rbf")
model_linear, model_rbf

(SVC(kernel='linear'), SVC())

## Part 6 - evaluate the models (5 points)

You will calculate and print precision, recall, and F-measure for the models on the test data. In `notes.md`, write down your comparison of these simple measures on the two models and any thoughts you might have on what they mean. (It could be very short, and since the samples do not stay stable between runs, you can save the evaluation scores in `notes.md` too.)

In [8]:
mc.eval_model(model_linear, test_X, test_y)
mc.eval_model(model_rbf, test_X, test_y)

SVC(kernel='linear') 
 Precision:0.6352941176470588 
 Recall:0.7311411992263056 
 F-measure:0.6798561151079136
SVC() 
 Precision:0.6332794830371568 
 Recall:0.758220502901354 
 F-measure:0.6901408450704225


## Part Bonus - try another sort of model from sklearn (5 points)

Write a separate, command-line script (not a notebook) uses `mycode.py` to do all of the above, except that it trains a non-SVM classifier model.  Any non-trivial model available in sklearn will do. Explain how to run your code and the results of your own evaluation in `notes.md`, including any observations or opinions you may have on the classifier method you used in comparison to SVM.

## Submission

Push to your fork of the GitHub repository (which must be made public) and submit the URL of your repository in Canvas.  You can submit this notebook with the output from your run, as long as you do not modify the code or text in it without permission from us.  